In [1]:
#ANMOL GUPTA
#20BCB0135
#SENTIMENT ANALYSIS FOR PRODUCT REVIEWS


In [2]:
import numpy as np
import bz2
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score,recall_score
from sklearn.model_selection import train_test_split

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.o

In [7]:
def convert_labels_text(file):
 labels = []
 texts = []
 for line in bz2.BZ2File(file):
     x = line.decode("utf-8")
     labels.append(int(x[9]) - 1)
     texts.append(x[10:].strip())
 return np.array(labels), texts
train_labels, train_texts = convert_labels_text('train.ft.txt.bz2')
test_labels, test_texts = convert_labels_text('test.ft.txt.bz2')

In [8]:
train_labels

array([1, 1, 1, ..., 0, 0, 1])

In [9]:
train_texts[2]

'Amazing!: This soundtrack is my favorite music of all time, hands down. The intense sadness of "Prisoners of Fate" (which means all the more if you\'ve played the game) and the hope in "A Distant Promise" and "Girl who Stole the Star" have been an important inspiration to me personally throughout my teen years. The higher energy tracks like "Chrono Cross ~ Time\'s Scar~", "Time of the Dreamwatch", and "Chronomantique" (indefinably remeniscent of Chrono Trigger) are all absolutely superb as well.This soundtrack is amazing music, probably the best of this composer\'s work (I haven\'t heard the Xenogears soundtrack, so I can\'t say for sure), and even if you\'ve never played the game, it would be worth twice the price to buy it.I wish I could give it 6 stars.'

In [10]:
train_texts=train_texts[0:2000]

In [13]:
train_labels=train_labels[0:2000]
train_texts=train_texts[0:2000]

In [14]:
train_labels.shape

(2000,)

In [15]:
len(train_texts)

2000

In [16]:
NON_ALPHANUM = re.compile(r'[\W]')
NON_ASCII = re.compile(r'[^a-z0-1\s]')
def normalize_texts(texts):
    normalized_texts = []
    for text in texts:
        lower = text.lower()
        no_punctuation = NON_ALPHANUM.sub(r' ', lower)
        no_non_ascii = NON_ASCII.sub(r'', no_punctuation)
        normalized_texts.append(no_non_ascii)
    return normalized_texts

train_texts = normalize_texts(train_texts)
test_texts = normalize_texts(test_texts)

In [17]:
train_texts[2]

'amazing   this soundtrack is my favorite music of all time  hands down  the intense sadness of  prisoners of fate   which means all the more if you ve played the game  and the hope in  a distant promise  and  girl who stole the star  have been an important inspiration to me personally throughout my teen years  the higher energy tracks like  chrono cross   time s scar     time of the dreamwatch   and  chronomantique   indefinably remeniscent of chrono trigger  are all absolutely superb as well this soundtrack is amazing music  probably the best of this composer s work  i haven t heard the xenogears soundtrack  so i can t say for sure   and even if you ve never played the game  it would be worth twice the price to buy it i wish i could give it  stars '

In [18]:
cv = CountVectorizer(binary=True)
cv.fit(train_texts)
X = cv.transform(train_texts)
X_test = cv.transform(test_texts)

In [19]:
X_test

<400000x12468 sparse matrix of type '<class 'numpy.int64'>'
	with 20049996 stored elements in Compressed Sparse Row format>

In [20]:
X_train, X_val, y_train, y_val = train_test_split(X, train_labels, train_size = 0.75)
lr = LogisticRegression(C = 0.05)
lr.fit(X_train, y_train)
rf = RandomForestClassifier(n_estimators=5)
rf.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:430: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  warnings.warn("Default solver will be changed to 'lbfgs' in 0.22. "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:489: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_store_unique_indices = np.zeros(y.shape, dtype=np.int)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated a

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=5,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [21]:
print("--------------Metrics for Logistic Regression--------------\n")
print(f"Accuracy: {accuracy_score(y_train, lr.predict(X_train))} ")
print(f"Confusion Matrix:\n{confusion_matrix(y_train, lr.predict(X_train))}")
print(f"Precision: {precision_score(y_train, lr.predict(X_train))} ")
print(f"Recall: {recall_score(y_train, lr.predict(X_train))} ")
print("\n--------------Metrics for Random Forest--------------\n")
print(f"Accuracy: {accuracy_score(y_train, rf.predict(X_train))} ")
print(f"Confusion Matrix:\n{confusion_matrix(y_train, rf.predict(X_train))}")
print(f"Precision: {precision_score(y_train, rf.predict(X_train))} ")
print(f"Recall: {recall_score(y_train, rf.predict(X_train))} ")


--------------Metrics for Logistic Regression--------------

Accuracy: 0.9513333333333334 
Confusion Matrix:
[[755  32]
 [ 41 672]]
Precision: 0.9545454545454546 
Recall: 0.9424964936886395 

--------------Metrics for Random Forest--------------

Accuracy: 0.9713333333333334 
Confusion Matrix:
[[770  17]
 [ 26 687]]
Precision: 0.9758522727272727 
Recall: 0.9635343618513323 


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\base.py:291: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  indices = (scores > 0).astype(np.int)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\base.py:291: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, 

In [22]:
#SO RANDOM FOREST IS BETTER SINCE IT AHS HIGHER ACCURACY SCORE